Lab 2 part 2
Group members:

Swati Nair (UBitName: swatishr; UB Person No: 50246994)

Bhavik Gala (UBIT Name: bhavikna; UB Person No: 50248608)

#Twitter Data Aggregation

In [3]:
import tweepy
from tweepy import OAuthHandler
import csv
import codecs
import time

1. Get the consumer and access Twitter keys
2. Get api object of class tweepy.API given the Auth keys

In [44]:
consumer_key = "0y8DKbzLfjEAVq2gSwH1jKbt1" 
consumer_secret = "l7XhQKwX8bK5ZAJ7eOJ2jYI2Fn33U9E3hqMpZzExcK2GX9Xo1L" 
access_token = "1471799647-jrMzb5XVAf8nuYoQH8CtoqFAuH8lxDMrt9L1V2X"
access_secret = "ik5TiBJZaU6p5RxTygJJkhOzVeR7CzVxiEXKS5YDwhMv9"

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

if (not api):
    print("Can't Authenticate")

1. Open CSV file to store the tweets

In [4]:
csvFile = open('./data/part2/guncontrol_tweets.csv','a+', encoding='utf-8', newline='')
csvWriter = csv.writer(csvFile)

1. For each tweet returned by tweepy.Cursor with search keyword = "gun control" and tweet_mode extended, get its full_text attribute
2. If it is a retweet, then get the retweeted_status.full_text attribute
3. Write it in the csv file
4. Close csv file

In [50]:
print("Started...")
for tweet in tweepy.Cursor(api.search,q="gun control",lang="en",tweet_mode="extended").items(50000):
    
    if (hasattr(tweet, 'retweeted_status')): 
        text = tweet.retweeted_status.full_text
    else:
        text = tweet.full_text

    csvWriter.writerow([tweet.user.name, text, tweet.created_at])
    
print("Done!!")

csvFile.close()

Started...
Rate limit reached. Sleeping for: 188
Rate limit reached. Sleeping for: 846
Rate limit reached. Sleeping for: 849
Rate limit reached. Sleeping for: 848
Rate limit reached. Sleeping for: 849
Rate limit reached. Sleeping for: 851
Rate limit reached. Sleeping for: 851
Rate limit reached. Sleeping for: 844
Rate limit reached. Sleeping for: 852
Rate limit reached. Sleeping for: 854
Rate limit reached. Sleeping for: 854
Rate limit reached. Sleeping for: 857
Rate limit reached. Sleeping for: 855
Rate limit reached. Sleeping for: 854
Rate limit reached. Sleeping for: 852
Rate limit reached. Sleeping for: 858
Rate limit reached. Sleeping for: 854
Rate limit reached. Sleeping for: 858
Done!!


#NYTimes Data Aggregation

1. Import articleAPI from nytimesarticle package
2. Store the api object with the generated NYTimes key

In [123]:
from nytimesarticle import articleAPI
api = articleAPI('f4335cdd509346a6a871a453caac1ece')

1. Search "gun control" using api.search for each date considering only 2 pages
2. Store the web_url in a csv file

In [124]:
search_term="gun control"
csvFile = open('./data/part2/guncontrol_nytimes.csv','a+', encoding='utf-8', newline='')
csvWriter = csv.writer(csvFile)

begin = [20180214, 20180301, 20180401]
end = [20180227, 20180330, 20180404]

for i in range(len(begin)):
    for nDate in range(begin[i],end[i]+1):
        for page in range(2):
            endDate = nDate+1
            articles = api.search(q = search_term, begin_date=nDate, end_date=endDate,page=page)
            items = articles['response']['docs']
            for item in items:
                # print(item, articles['response']['docs'][item]['web_url'])
                csvWriter.writerow([item['web_url']])
            time.sleep(1)
        
begin = [20180228, 20180331]
end = [20180301, 20180401]

for i in range(len(begin)):
    for page in range(2):
        articles = api.search(q = search_term, begin_date=begin[i], end_date=end[i],page=page)
        items = articles['response']['docs']
        for item in items:
            # print(item, articles['response']['docs'][item]['web_url'])
            csvWriter.writerow([item['web_url']])
        time.sleep(1)

print("Done!!")
csvFile.close()

Done!!


NyTimes web scraper
1. For each web_url read from csv, create a new file (named as index_num.txt) per article
2. Parse the web page text using BeautifulSoup
3. Save the header tag (h1)
4. Consider the main story body paragraph tags excluding all other irrelevant p classes as given in not_included dictionary

In [47]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import requests
import time

index = 0

with open("./data/part2/guncontrol_nytimes.csv", "r", encoding='utf-8',newline="") as f:
    reader = csv.reader(f)
    for url in reader:
        f = open('./data/part2/nytimes_data/'+str(index)+'.txt','w+', encoding='utf-8')
        print(index, url)
        client = requests.get(url[0])
        html_page = client.text
        client.close()

        page_soup = soup(html_page, "html.parser")

        headline = page_soup.find_all("h1")
        f.write(headline[0].get_text()+"\n")

        not_included = {"byline-dateline","view-all-link","error","story-print-citation","feedback-message","user-action"};
        for para in page_soup.select("p"):
            try:
                if any(x in not_included for x in para["class"]):
                    continue
            except KeyError:
                pass
            f.write(para.get_text())
        index+=1
        f.close()
        time.sleep(1)

f.close()

151 ['https://www.nytimes.com/2018/02/23/us/politics/fact-check-mass-shootings-gun-laws.html']
152 ['https://www.nytimes.com/2018/02/23/us/politics/australia-gun-crackdown-trump-school-shooting.html']
153 ['https://www.nytimes.com/2018/02/23/us/politics/trump-video-games-shootings.html']
154 ['https://www.nytimes.com/2018/02/23/us/red-flag-laws-guns.html']
155 ['https://www.nytimes.com/2018/02/22/us/politics/gun-background-checks-prosecutions.html']
156 ['https://www.nytimes.com/2018/02/22/us/politics/gun-background-checks-prosecutions.html']
157 ['https://www.nytimes.com/2018/02/23/opinion/parkland-florida-gun-control.html']
158 ['https://www.nytimes.com/2018/02/23/podcasts/the-daily/gun-access-mentally-ill.html']
159 ['https://www.nytimes.com/2018/02/23/opinion/brian-mast-assault-weapons-ban.html']
160 ['https://www.nytimes.com/2018/02/22/opinion/teachers-guns-classrooms.html']
161 ['https://www.nytimes.com/2018/02/23/books/review/ismail-kadare-a-girl-in-exile.html']
162 ['https://ww

243 ['https://www.nytimes.com/interactive/2018/03/02/world/international-gun-laws.html']
244 ['https://www.nytimes.com/2018/03/02/opinion/go-ahead-millennials-destroy-us.html']
245 ['https://www.nytimes.com/2018/03/02/business/delta-nra-discount.html']
246 ['https://www.nytimes.com/2018/03/02/podcasts/the-daily/tom-rooney-trump-washington.html']
247 ['https://www.nytimes.com/2018/03/02/us/central-michigan-campus-shooting.html']
248 ['https://www.nytimes.com/2018/03/01/arts/jimmy-kimmel-oscars-metoo.html']
249 ['https://www.nytimes.com/2018/03/02/briefing/nra-tariffs-noreaster.html']
250 ['https://www.nytimes.com/2018/03/02/learning/lesson-plans/teenagers-in-the-times-february-2018.html']
251 ['https://www.nytimes.com/2018/03/02/nyregion/william-petit-home-invasion-connecticut-legislature.html']
252 ['https://www.nytimes.com/2018/03/02/briefing/putin-snow-italy-election.html']
253 ['https://www.nytimes.com/2018/03/01/us/politics/trump-chaos-oval-office.html']
254 ['https://www.nytimes.c

333 ['https://www.nytimes.com/2018/03/08/us/daylight-saving-time-florida.html']
334 ['https://www.nytimes.com/2018/03/08/arts/television/alan-cumming-instinct-cbs.html']
335 ['https://www.nytimes.com/2018/03/08/business/dealbook/cigna-express-scripts-deal.html']
336 ['https://www.nytimes.com/2018/03/08/us/immigration-california-trump-sessions.html']
337 ['https://www.nytimes.com/2018/03/07/us/politics/jeff-sessions-california-lawsuit.html']
338 ['https://www.nytimes.com/2018/03/08/learning/what-did-you-read-watch-or-listen-to-in-the-times-this-week.html']
339 ['https://www.nytimes.com/2018/03/08/learning/whats-going-on-in-this-graph-march-13-2018.html']
340 ['https://www.nytimes.com/2018/03/07/science/thomas-jessell-dismissed.html']
341 ['https://www.nytimes.com/interactive/2018/obituaries/overlooked-henrietta-lacks.html']
342 ['https://www.nytimes.com/interactive/2018/obituaries/overlooked-nella-larsen.html']
343 ['https://www.nytimes.com/2018/03/07/theater/review-hello-from-the-child

423 ['https://www.nytimes.com/2018/03/14/us/teacher-gun-accidental-discharge.html']
424 ['https://www.nytimes.com/2018/03/14/us/photos-gun-protest.html']
425 ['https://www.nytimes.com/video/us/100000005794215/students-are-walking-out-to-protest-gun-violence-heres-their-videos.html']
426 ['https://www.nytimes.com/2018/03/14/arts/design/asia-week-objects-galleries-asia-society-japan-society.html']
427 ['https://www.nytimes.com/2018/03/14/briefing/walkout-conor-lamb-russia.html']
428 ['https://www.nytimes.com/2018/03/14/briefing/stephen-hawking-theresa-may-school-walkout.html']
429 ['https://www.nytimes.com/2018/03/14/us/pennsylvania-election-lamb.html']
430 ['https://www.nytimes.com/2018/03/14/us/politics/andrew-mccabe-fbi-firing-recommendation-justice-department.html']
431 ['https://www.nytimes.com/2018/03/14/us/politics/democrats-republicans-pennsylvania-special-election.html']
432 ['https://www.nytimes.com/2018/03/14/theater/harper-lee-estate-lawsuit-broadway-mockingbird.html']
433 ['

512 ['https://www.nytimes.com/interactive/2018/03/20/learning/20StudentNewsQuiz-Hawking-Basketball-Protests.html']
513 ['https://www.nytimes.com/2018/03/19/nyregion/cynthia-nixon-cuomo-governor.html']
514 ['https://www.nytimes.com/2018/03/20/us/politics/betsy-devos-congress-racial-disparities.html']
515 ['https://www.nytimes.com/2018/03/20/briefing/vladimir-putin-facebook-white-rhino.html']
516 ['https://www.nytimes.com/2018/03/20/books/pence-gay-bunny-bundo.html']
517 ['https://www.nytimes.com/2018/03/20/movies/monte-review.html']
518 ['https://www.nytimes.com/2018/03/20/opinion/israel-palestinians-two-states.html']
519 ['https://www.nytimes.com/2018/03/20/obituaries/francis-bator-influential-white-house-economist-dies-at-92.html']
520 ['https://www.nytimes.com/2018/03/20/science/robert-langlands-abel-prize-mathematics.html']
521 ['https://www.nytimes.com/2018/03/20/us/politics/supreme-court-abortion-free-speech-crisis-pregnancy-centers.html']
522 ['https://www.nytimes.com/2018/03/20/

602 ['https://www.nytimes.com/aponline/2018/03/24/us/politics/ap-us-student-gun-protests-the-latest.html']
603 ['https://www.nytimes.com/aponline/2018/03/24/us/politics/ap-us-student-gun-protests-vignettes-.html']
604 ['https://www.nytimes.com/aponline/2018/03/24/us/ap-us-student-gun-protests-six-minutes.html']
605 ['https://www.nytimes.com/2018/03/25/us/gun-march-organizers.html']
606 ['https://www.nytimes.com/reuters/2018/03/25/world/asia/25reuters-usa-guns.html']
607 ['https://www.nytimes.com/2018/03/25/briefing/john-bolton-gun-control-china.html']
608 ['https://www.nytimes.com/2018/03/25/briefing/gun-control-john-bolton-facebook.html']
609 ['https://www.nytimes.com/2018/03/24/us/emma-gonzalez-march-for-our-lives.html']
610 ['https://www.nytimes.com/2018/03/25/nyregion/new-york-state-budget-deadline.html']
611 ['https://www.nytimes.com/2018/03/25/us/politics/guns-midterms-republicans.html']
612 ['https://www.nytimes.com/video/us/100000005817208/fiery-speech-and-charged-silence-from-

692 ['https://www.nytimes.com/reuters/2018/03/29/technology/29reuters-telegram-app-outages.html']
693 ['https://www.nytimes.com/aponline/2018/03/29/us/ap-us-fabolous-domestic-violence-charged.html']
694 ['https://www.nytimes.com/aponline/2018/03/29/us/ap-us-fatal-standoff-south-carolina.html']
695 ['https://www.nytimes.com/reuters/2018/03/29/business/29reuters-constellation-results.html']
696 ['https://www.nytimes.com/reuters/2018/03/29/business/29reuters-dollarama-results.html']
697 ['https://www.nytimes.com/reuters/2018/03/29/business/29reuters-cmc-markets-outlook.html']
698 ['https://www.nytimes.com/2018/03/29/books/fantastic-four-marvel-return.html']
699 ['https://www.nytimes.com/2018/03/29/nyregion/connecticut-governor-ganim-bridgeport.html']
700 ['https://www.nytimes.com/2018/03/29/theater/5-shows-new-york-saint-joan-iceman-cometh.html']
701 ['https://www.nytimes.com/2018/03/29/arts/john-legend-and-the-jesus-christ-superstar-cast-on-faith-and-musicals.html']
702 ['https://www.nyt

778 ['https://www.nytimes.com/2018/04/04/magazine/judge-john-hodgman-on-a-wannabe-astronaut.html']
779 ['https://www.nytimes.com/reuters/2018/04/04/business/04reuters-mothercare-ceo.html']
780 ['https://www.nytimes.com/reuters/2018/04/04/business/04reuters-topps-tiles-outlook.html']
781 ['https://www.nytimes.com/reuters/2018/04/03/sports/03reuters-games-commonwealth-rugby-sevens-australia.html']
782 ['https://www.nytimes.com/2018/04/04/opinion/critiques-trump.html']
783 ['https://www.nytimes.com/reuters/2018/04/04/sports/soccer/04reuters-soccer-asia.html']
784 ['https://www.nytimes.com/2018/04/04/nyregion/homework-therapists-tutoring-counseling-new-york.html']
785 ['https://www.nytimes.com/2018/04/04/arts/television/jimmy-kimmel-post-office-amazon.html']
786 ['https://www.nytimes.com/reuters/2018/04/04/sports/soccer/04reuters-soccer-asia-women-asiancup.html']
787 ['https://www.nytimes.com/reuters/2018/04/04/business/04reuters-hyundai-mobis-elliott.html']
788 ['https://www.nytimes.com/2

In [9]:
f.close()